In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import tensorflow
import keras

In [6]:
with open("datasets/processed/vectorized_scripts_df.pickle", "rb") as r:
    dataframe = pickle.load(r)

In [7]:
models = {}
with open("models/decision_tree.model", "rb") as r:
    models["decision_tree"] = pickle.load(r)
with open("models/MLP.model", "rb") as r:
    models["MLP"] = pickle.load(r)
with open("models/random_forest.model", "rb") as r:
    models["random_forest"] = pickle.load(r)
with open("models/logistic_regression.model", "rb") as r:
    models["logistic_regression"] = pickle.load(r)
with open("models/k_nearest_neighbors.model", "rb") as r:
    models["k_n_n"] = pickle.load(r)
models["rnn LSTM"] = keras.models.load_model(
            '/Users/fabio/Documents/Progetto IR/models/RNNLSTM')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-02 10:13:15.047623: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-02 10:13:15.047781: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
all_pad_vec = np.array([list(x) for x in np.array(dataframe['pad_vectorized_text'])])

In [9]:
for k,v in models.items():
    if(k == 'rnn LSTM'):
        dataframe[k] = list(v.predict(all_pad_vec))
    else:
        dataframe[k] = v.predict(list(all_pad_vec))

2022-12-02 10:15:46.700569: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-02 10:15:47.214452: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:15:47.395879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:15:47.412542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:15:48.315660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:15:48.327710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-02 10:15:49.187163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114

9514/9514 [==============================] - 2576s 270ms/step


In [10]:
def from_categorical(vec):
        new_vec = []
        for v in vec:
            max = v[0]
            index = 0
            for i, val in enumerate(v):
                if (val > max):
                    max = val
                    index = i
            new_vec.append(index)
        return new_vec

In [11]:
numeric_predictions_LSTM = from_categorical(dataframe['rnn LSTM'].values)
dataframe['rnn LSTM'] = numeric_predictions_LSTM


In [12]:
from classes.cleaner import Cleaner
from classes.vectorizer import Vectorizer

In [13]:
C = Cleaner()
V = Vectorizer()

In [14]:
dataframe.drop(["pad_vectorized_text"], axis=1, inplace=True)

In [15]:
dataframe.head(10)

,id,conversation_id,meta.movie_idx_x,speaker,meta.character_name,meta.gender,reply_to,text,cleaned_text,decision_tree,MLP,random_forest,logistic_regression,k_n_n,rnn LSTM
0,L1045,L1044,m0,u0,BIANCA,f,L1044,They do not!,they do not,6,4,4,4,4,4
1,L985,L984,m0,u0,BIANCA,f,L984,I hope so.,i hope so,3,4,8,4,8,4
2,L925,L924,m0,u0,BIANCA,f,L924,Let's go.,let us go,6,4,4,4,4,4
3,L872,L870,m0,u0,BIANCA,f,L871,Okay -- you're gonna need to learn how to lie.,okay you re go to need to learn how to lie,3,8,4,4,5,4
4,L870,L870,m0,u0,BIANCA,f,None,I'm kidding. You know how sometimes you just ...,i am kid you know how sometimes you just becom...,1,8,4,8,4,4
5,L869,L866,m0,u0,BIANCA,f,L868,Like my fear of wearing pastels?,like my fear of wear pastel,4,4,4,4,1,4
6,L867,L866,m0,u0,BIANCA,f,L866,What good stuff?,what good stuff,0,4,8,4,6,4
7,L864,L862,m0,u0,BIANCA,f,L863,"Me. This endless ...blonde babble. I'm like, ...",me this endless blonde babble i am like bore m...,2,8,4,4,6,4
8,L862,L862,m0,u0,BIANCA,f,None,do you listen to this crap?,do you listen to this crap,6,8,4,4,4,4
9,L860,L860,m0,u0,BIANCA,f,None,"Then Guillermo says, ""If you go any lighter, y...",then guillermo say if you go any light you re ...,1,3,8,1,1,4


In [16]:
with open("datasets/processed/script_dataframe_emotion.pickle", "wb") as r:
    pickle.dump(dataframe, r)